In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input, LSTM, Add, Average
from tensorflow.keras.metrics import Recall,Precision
from sklearn.metrics import f1_score

In [2]:
asos_df = pd.read_csv('/Volumes/ESD-ISO/project_preson/shared_data_preparation/ASOS_data_preprocessed_V4.csv')

In [3]:
# rain_tomorrow 열을 rainy_day 열에서 12시간 이후 값을 가져와 생성
asos_df['rain_tomorrow'] = asos_df['rainy_day'].shift(-12)

In [4]:
asos_df.drop(index=asos_df[asos_df['rain_tomorrow'].isna()].index, inplace=True) # 12시간 뒤에 데이터가 없는 마지막 12개열 삭제

In [5]:
encoder = LabelEncoder()
asos_df.clfmAbbrCd = encoder.fit_transform(asos_df.clfmAbbrCd)

In [6]:
asos_df['tm'] = pd.to_datetime(asos_df['tm'])

In [7]:
base_time = asos_df['tm'].iloc[0]
asos_df['tm'] = (asos_df['tm']-base_time).dt.total_seconds()

In [8]:
asos_df['tm'].value_counts()

0.000000e+00    3
1.028052e+08    3
1.027404e+08    3
1.027512e+08    3
1.027620e+08    3
               ..
4.694436e+08    1
4.694328e+08    1
4.694220e+08    1
4.694112e+08    1
1.420020e+09    1
Name: tm, Length: 95248, dtype: int64

In [9]:
# 'group' 컬럼 생성: 24개 단위로 그룹 ID 부여
asos_df['group'] = np.arange(len(asos_df)) // 24

# rain_tomorrow는 별도로 처리
rain_tomorrow_grouped = asos_df.groupby('group')['rain_tomorrow'].apply(list)

# 나머지 컬럼은 24개 단위로 묶기
other_columns_grouped = asos_df.groupby('group')[['tm','hm', 'pv', 'td', 'pa', 'm005Te', 'm01Te', 'm02Te', 'm03Te', 'clfmAbbrCd', 'lcsCh']].apply(lambda x: x.values.tolist())

In [10]:
y_data = []
for rain_tomorrow in rain_tomorrow_grouped:
    if 1 in rain_tomorrow:
        y_data.append(1)
    else:
        y_data.append(0)

In [11]:
np.array(y_data).shape, np.vstack(other_columns_grouped.tolist()).shape

((4729,), (113496, 11))

In [12]:
data_X = np.vstack(other_columns_grouped.iloc[:-1].tolist())

In [13]:
data_X[0]

array([ 0.0000e+00,  5.6000e+01,  3.9000e+00, -6.0000e+00,  1.0159e+03,
       -1.2000e+00,  3.0000e-01,  5.0000e-01,  6.0000e-01,  2.6000e+01,
        0.0000e+00])

In [14]:
data_y = np.array(y_data[:-1])

In [15]:
scaler = MinMaxScaler()
data_X = scaler.fit_transform(data_X).reshape(-1,24,11)

In [16]:
spilt_index = round(len(data_X)/10)*7

In [17]:
train_X = data_X[:spilt_index]
train_y = data_y[:spilt_index]
test_X = data_X[spilt_index:]
test_y = data_y[spilt_index:]

In [18]:
train_X.shape,train_y.shape,test_X.shape,test_y.shape

((3311, 24, 11), (3311,), (1417, 24, 11), (1417,))

In [19]:
def lstm_model_V1():

    inputs = Input(shape=(24, 11))

    layer_1 = LSTM(units=32, return_sequences=True)(inputs)

    layer_2 = LSTM(units=64, return_sequences=False)(layer_1) 

    D_1 = Dense(units=64, activation='relu')(layer_2)
    D_2 = Dense(units=128, activation='relu')(D_1)
    D_3 = Dense(units=64, activation='relu')(D_2)
    D_4 = Dense(units=32, activation='relu')(D_3)
    D_5 = Dense(units=16, activation='relu')(D_4)
    outputs = Dense(units=1, activation='sigmoid')(D_5)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [20]:
model = lstm_model_V1()
model.summary()

Metal device set to: Apple M2 Pro


2025-01-17 15:13:10.192015: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-17 15:13:10.192127: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 24, 11)]          0         
                                                                 
 lstm (LSTM)                 (None, 24, 32)            5632      
                                                                 
 lstm_1 (LSTM)               (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 32)                2080  

In [21]:
model.fit(train_X, train_y, validation_split=0.2, epochs=100)

Epoch 1/100


2025-01-17 15:13:10.610934: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2025-01-17 15:13:11.316597: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-17 15:13:11.467612: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-17 15:13:11.595142: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-17 15:13:11.906444: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-17 15:13:12.188269: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


83/83 [==============================] - ETA: 0s - loss: 0.6753 - accuracy: 0.5759

2025-01-17 15:13:14.592111: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-17 15:13:14.651789: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-17 15:13:14.756293: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


83/83 [==============================] - 5s 36ms/step - loss: 0.6753 - accuracy: 0.5759 - val_loss: 0.7066 - val_accuracy: 0.4027
Epoch 2/100
83/83 [==============================] - 2s 18ms/step - loss: 0.6612 - accuracy: 0.5948 - val_loss: 0.6619 - val_accuracy: 0.6591
Epoch 3/100
83/83 [==============================] - 2s 18ms/step - loss: 0.6471 - accuracy: 0.6125 - val_loss: 0.6693 - val_accuracy: 0.6048
Epoch 4/100
83/83 [==============================] - 2s 18ms/step - loss: 0.6390 - accuracy: 0.6186 - val_loss: 0.6507 - val_accuracy: 0.6214
Epoch 5/100
83/83 [==============================] - 2s 19ms/step - loss: 0.6281 - accuracy: 0.6337 - val_loss: 0.6642 - val_accuracy: 0.6621
Epoch 6/100
83/83 [==============================] - 2s 19ms/step - loss: 0.6198 - accuracy: 0.6405 - val_loss: 0.6433 - val_accuracy: 0.6787
Epoch 7/100
83/83 [==============================] - 2s 19ms/step - loss: 0.6210 - accuracy: 0.6424 - val_loss: 0.6465 - val_accuracy: 0.6606
Epoch 8/100
83/83 

83/83 [==============================] - 2s 18ms/step - loss: 0.3965 - accuracy: 0.8014 - val_loss: 1.0679 - val_accuracy: 0.4977
Epoch 59/100
83/83 [==============================] - 2s 18ms/step - loss: 0.3961 - accuracy: 0.8036 - val_loss: 1.0587 - val_accuracy: 0.5128
Epoch 60/100
83/83 [==============================] - 2s 18ms/step - loss: 0.3780 - accuracy: 0.8055 - val_loss: 1.1617 - val_accuracy: 0.4661
Epoch 61/100
83/83 [==============================] - 2s 18ms/step - loss: 0.3803 - accuracy: 0.8146 - val_loss: 1.1178 - val_accuracy: 0.5008
Epoch 62/100
83/83 [==============================] - 2s 18ms/step - loss: 0.3675 - accuracy: 0.8236 - val_loss: 1.1272 - val_accuracy: 0.5068
Epoch 63/100
83/83 [==============================] - 2s 18ms/step - loss: 0.3304 - accuracy: 0.8391 - val_loss: 1.3780 - val_accuracy: 0.4555
Epoch 64/100
83/83 [==============================] - 2s 18ms/step - loss: 0.3585 - accuracy: 0.8187 - val_loss: 1.1927 - val_accuracy: 0.4887
Epoch 65/100

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(test_X,test_y)
print(f"테스트 정확도: {accuracy*100:.2f}%")

In [ ]:
y_hat = model.predict(test_X)

In [ ]:
y_hat = y_hat.argmax(axis=1)

In [ ]:
f1 = f1_score(test_y, y_hat, average='micro')
print(f"F1 스코어: {f1:.4f}")

In [ ]:
def lstm_model_V2():

    inputs = Input(shape=(24, 11))

    layer_1_tensor = LSTM(units=32, return_sequences=True)(inputs)
    
    layer_2_1 = LSTM(units=32, return_sequences=True)(layer_1_tensor)
    layer_2_2 = LSTM(units=32, return_sequences=True)(inputs)
    second_layer = Add()([layer_2_1, layer_2_2])
    dropout_1 = Dropout(0.2)(second_layer)
    
    layer_3_tensor = LSTM(units=128, return_sequences=True)(dropout_1)
    dropout_2 = Dropout(0.2)(layer_3_tensor)
    
    layer_4_1_tensor = LSTM(units=64, return_sequences=False)(dropout_2)
    layer_4_2_tensor = LSTM(units=64, return_sequences=False)(layer_1_tensor)
    last_layer = Average()([layer_4_1_tensor, layer_4_2_tensor]) 
    dropout_3 = Dropout(0.2)(last_layer)
    
    D_1 = Dense(units=64, activation='relu')(dropout_3)
    D_2 = Dense(units=32, activation='relu')(D_1)
    dropout_4 = Dropout(0.2)(D_2)
    
    outputs = Dense(units=1, activation='sigmoid')(dropout_4)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',Recall(),Precision()])
    return model

In [ ]:
model = lstm_model_V2()

model.summary()

In [ ]:
model.fit(train_X, train_y, validation_split=0.2, epochs=50)

In [ ]:
loss, accuracy,_,_ = model.evaluate(test_X, test_y)
print(f"테스트 정확도: {accuracy*100:.2f}%")

In [ ]:
y_hat = model.predict(test_X)

In [ ]:
y_hat = y_hat.argmax(axis=1)

In [ ]:
f1 = f1_score(test_y, y_hat, average='micro')
print(f"F1 스코어: {f1:.4f}")

## 컬럼 제거버전 데이터

In [154]:
asos_df = pd.read_csv('/Volumes/ESD-ISO/project_preson/shared_data_preparation/ASOS_1980_2024_After_drop.csv')

In [155]:
# hours_12 열을 rain 열에서 12시간 이후 값을 가져와 생성
asos_df['hours_12'] = asos_df['rain'].shift(-12)

In [156]:
asos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127394 entries, 0 to 127393
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   tm          127394 non-null  object 
 1   hm          127392 non-null  float64
 2   pv          127394 non-null  float64
 3   td          127392 non-null  float64
 4   pa          127394 non-null  float64
 5   clfmAbbrCd  127394 non-null  object 
 6   lcsCh       127394 non-null  float64
 7   m005Te      127392 non-null  float64
 8   m01Te       127392 non-null  float64
 9   m02Te       127392 non-null  float64
 10  m03Te       127392 non-null  float64
 11  rain        127394 non-null  int64  
 12  hours_12    127382 non-null  float64
dtypes: float64(10), int64(1), object(2)
memory usage: 12.6+ MB


In [157]:
asos_df.drop(index=asos_df[asos_df['hours_12'].isna()].index, inplace=True) # 12시간 뒤에 데이터가 없는 마지막 12개열 삭제

In [158]:
encoder = LabelEncoder()
asos_df.clfmAbbrCd = encoder.fit_transform(asos_df.clfmAbbrCd)

In [159]:
asos_df['tm'] = pd.to_datetime(asos_df['tm'])

In [160]:
base_time = asos_df['tm'].iloc[0]
asos_df['tm'] = (asos_df['tm']-base_time).dt.total_seconds()

In [161]:
asos_df['tm'].value_counts()

0.000000e+00    2
9.050400e+07    2
9.044640e+07    2
9.045000e+07    2
9.045360e+07    2
               ..
1.294337e+09    1
1.294340e+09    1
1.294344e+09    1
1.294348e+09    1
1.420027e+09    1
Name: tm, Length: 89790, dtype: int64

In [162]:
# 'group' 컬럼 생성: 24개 단위로 그룹 ID 부여
asos_df['group'] = np.arange(len(asos_df)) // 24

asos_df

,tm,hm,pv,td,pa,clfmAbbrCd,lcsCh,m005Te,m01Te,m02Te,m03Te,rain,hours_12,group
0,0.000000e+00,NaN,3.700757,NaN,1013.876282,72,0.0,NaN,NaN,NaN,NaN,0,0.0,0
1,3.600000e+03,NaN,3.700757,NaN,1013.876282,72,0.0,NaN,NaN,NaN,NaN,0,0.0,0
2,7.200000e+03,56.000000,3.900000,-6.0,1015.900000,26,0.0,-1.200000,0.300000,0.500000,0.6,0,0.0,0
3,1.080000e+04,58.333333,3.700757,-5.6,1013.876282,72,0.0,-1.083333,0.316667,0.483333,0.6,0,0.0,0
4,1.440000e+04,60.666667,3.700757,-5.2,1013.876282,72,0.0,-0.966667,0.333333,0.466667,0.6,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127377,1.420013e+09,76.000000,5.700000,-1.0,1009.100000,51,15.0,-0.200000,-0.300000,0.600000,1.6,0,0.0,5307
127378,1.420016e+09,73.000000,5.900000,-0.4,1008.900000,51,16.0,-0.200000,-0.300000,0.600000,1.6,0,0.0,5307
127379,1.420020e+09,71.000000,6.500000,0.8,1008.700000,51,12.0,-0.200000,-0.300000,0.600000,1.6,0,0.0,5307
127380,1.420024e+09,61.000000,7.100000,2.1,1007.600000,51,13.0,-0.200000,-0.300000,0.600000,1.5,0,0.0,5307


In [163]:
sum(asos_df.groupby(['group'])['hours_12'].sum() == 0)

3840

In [164]:
# 예제 DataFrame 생성
# asos_df는 이미 존재하는 것으로 가정

# 조건에 맞는 그룹을 찾기
group_sums = asos_df.groupby('group')['hours_12'].sum()

# 조건에 맞는 그룹 (sum == 0)
groups_to_delete = group_sums[group_sums == 0].index

# 3000개만 선택 (최대)
groups_to_delete = groups_to_delete[:2500]

# 조건에 맞는 그룹 제거
filtered_df = asos_df[~asos_df['group'].isin(groups_to_delete)]

# 결과 확인
print("삭제 전 그룹 수:", asos_df['group'].nunique())
print("삭제 후 그룹 수:", filtered_df['group'].nunique())

삭제 전 그룹 수: 5308
삭제 후 그룹 수: 2808


In [165]:
filtered_df

,tm,hm,pv,td,pa,clfmAbbrCd,lcsCh,m005Te,m01Te,m02Te,m03Te,rain,hours_12,group
1152,4.147200e+06,76.000000,3.700757,-6.633333,1013.876282,72,0.0,-2.400000,-1.0,-1.033333,-1.466667,0,0.0,48
1153,4.150800e+06,78.000000,3.700757,-6.866667,1013.876282,72,0.0,-2.950000,-1.2,-1.016667,-1.433333,0,0.0,48
1154,4.154400e+06,80.000000,3.600000,-7.100000,1019.700000,72,0.0,-3.500000,-1.4,-1.000000,-1.400000,0,0.0,48
1155,4.158000e+06,79.666667,3.700757,-7.150000,1013.876282,72,0.0,-3.383333,-1.5,-1.116667,-1.450000,0,0.0,48
1156,4.161600e+06,79.333333,3.700757,-7.200000,1013.876282,72,0.0,-3.266667,-1.6,-1.233333,-1.500000,0,0.0,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127377,1.420013e+09,76.000000,5.700000,-1.000000,1009.100000,51,15.0,-0.200000,-0.3,0.600000,1.600000,0,0.0,5307
127378,1.420016e+09,73.000000,5.900000,-0.400000,1008.900000,51,16.0,-0.200000,-0.3,0.600000,1.600000,0,0.0,5307
127379,1.420020e+09,71.000000,6.500000,0.800000,1008.700000,51,12.0,-0.200000,-0.3,0.600000,1.600000,0,0.0,5307
127380,1.420024e+09,61.000000,7.100000,2.100000,1007.600000,51,13.0,-0.200000,-0.3,0.600000,1.500000,0,0.0,5307


In [166]:
# hours_12 별도로 처리
hours_12_grouped = filtered_df.groupby('group')['hours_12'].apply(list)

# 나머지 컬럼은 24개 단위로 묶기
other_columns_grouped = filtered_df.groupby('group')[['hm', 'td', 'pa', 'm005Te', 'm01Te', 'm02Te', 'm03Te', 'clfmAbbrCd', 'lcsCh']].apply(lambda x: x.values.tolist())

In [167]:
other_columns_grouped

group
48      [[76.0, -6.633333333333333, 1013.8762815249268...
94      [[71.33333333333333, 6.166666666666667, 1005.3...
95      [[67.66666666666667, 12.266666666666666, 1005....
96      [[91.66666666666669, 10.133333333333333, 998.7...
101     [[62.333333333333336, 1.333333333333333, 1005....
                              ...                        
5303    [[72.0, -0.4, 1011.3, -0.1, -0.3, 1.0, 2.0, 72...
5304    [[54.0, -12.7, 1016.5, -0.2, -0.3, 1.1, 2.0, 7...
5305    [[57.0, -10.9, 1013.4, -0.3, -0.4, 0.9, 1.9, 7...
5306    [[56.0, -10.0, 1013.8, -0.7, -0.7, 0.8, 1.8, 5...
5307    [[69.0, -3.0, 1010.8, -0.2, -0.4, 0.7, 1.6, 51...
Length: 2808, dtype: object

In [168]:
y_data = []
for hours_12 in hours_12_grouped:
    if 1 in hours_12:
        y_data.append(1)
    else:
        y_data.append(0)

In [169]:
np.array(y_data).shape, np.vstack(other_columns_grouped.tolist()).shape

((2808,), (67382, 9))

In [170]:
asos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127382 entries, 0 to 127381
Data columns (total 14 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   tm          127382 non-null  float64
 1   hm          127380 non-null  float64
 2   pv          127382 non-null  float64
 3   td          127380 non-null  float64
 4   pa          127382 non-null  float64
 5   clfmAbbrCd  127382 non-null  int64  
 6   lcsCh       127382 non-null  float64
 7   m005Te      127380 non-null  float64
 8   m01Te       127380 non-null  float64
 9   m02Te       127380 non-null  float64
 10  m03Te       127380 non-null  float64
 11  rain        127382 non-null  int64  
 12  hours_12    127382 non-null  float64
 13  group       127382 non-null  int64  
dtypes: float64(11), int64(3)
memory usage: 13.6 MB


In [171]:
data_X = np.vstack(other_columns_grouped.iloc[:-1].tolist())

In [172]:
data_y = np.array(y_data[:-1])

In [173]:
unique_values = np.unique(data_y)

# 각 고유값의 개수 계산
counts = [np.count_nonzero(data_y == value) for value in unique_values]

# 출력
for value, count in zip(unique_values, counts):
    print(f"값: {value}, 개수: {count}")

값: 0, 개수: 1339
값: 1, 개수: 1468


In [178]:
scaler = MinMaxScaler()
data_X = scaler.fit_transform(data_X.reshape(-1,1)).reshape(-1,24,9)

In [179]:
spilt_index = round(len(data_X)/10)*7

In [180]:
train_X = data_X[:spilt_index]
train_y = data_y[:spilt_index]
test_X = data_X[spilt_index:]
test_y = data_y[spilt_index:]

In [181]:
train_X.shape,train_y.shape,test_X.shape,test_y.shape

((1967, 24, 9), (1967,), (840, 24, 9), (840,))

In [182]:
def lstm_model_V1():

    inputs = Input(shape=(24, 11))

    layer_1 = LSTM(units=32, return_sequences=True)(inputs)

    layer_2 = LSTM(units=64, return_sequences=False)(layer_1) 

    D_1 = Dense(units=64, activation='relu')(layer_2)
    D_2 = Dense(units=128, activation='relu')(D_1)
    D_3 = Dense(units=64, activation='relu')(D_2)
    D_4 = Dense(units=32, activation='relu')(D_3)
    D_5 = Dense(units=16, activation='relu')(D_4)
    outputs = Dense(units=1, activation='sigmoid')(D_5)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [184]:
def lstm_model_V2():

    inputs = Input(shape=(24, 9))

    layer_1_tensor = LSTM(units=32, return_sequences=True)(inputs)
    
    layer_2_1 = LSTM(units=32, return_sequences=True,
                     kernel_initializer='glorot_uniform',  # 가중치 초기화
                     recurrent_initializer='orthogonal',  # 순환 가중치 초기화
                     bias_initializer='zeros')(layer_1_tensor)
    layer_2_2 = LSTM(units=32, return_sequences=True,
                     kernel_initializer='glorot_uniform',  # 가중치 초기화
                     recurrent_initializer='orthogonal',  # 순환 가중치 초기화
                     bias_initializer='zeros')(inputs)
    second_layer = Add()([layer_2_1, layer_2_2])
    dropout_1 = Dropout(0.2)(second_layer)
    
    layer_3_tensor = LSTM(units=128, return_sequences=True,
                         kernel_initializer='glorot_uniform',  # 가중치 초기화
                         recurrent_initializer='orthogonal',  # 순환 가중치 초기화
                         bias_initializer='zeros')(dropout_1)
    dropout_2 = Dropout(0.2)(layer_3_tensor)
    
    layer_4_1_tensor = LSTM(units=64, return_sequences=False,
                             kernel_initializer='glorot_uniform',  # 가중치 초기화
                             recurrent_initializer='orthogonal',  # 순환 가중치 초기화
                             bias_initializer='zeros')(dropout_2)
    layer_4_2_tensor = LSTM(units=64, return_sequences=False,
                             kernel_initializer='glorot_uniform',  # 가중치 초기화
                             recurrent_initializer='orthogonal',  # 순환 가중치 초기화
                             bias_initializer='zeros')(layer_1_tensor)
    last_layer = Average()([layer_4_1_tensor, layer_4_2_tensor]) 
    dropout_3 = Dropout(0.2)(last_layer)
    
    D_1 = Dense(units=64, activation='relu')(dropout_3)
    D_2 = Dense(units=32, activation='relu')(D_1)
    dropout_4 = Dropout(0.2)(D_2)
    
    outputs = Dense(units=1, activation='sigmoid')(dropout_4)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',Recall(),Precision()])
    return model

In [185]:
model = lstm_model_V2()

model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 24, 9)]      0           []                               
                                                                                                  
 lstm_42 (LSTM)                 (None, 24, 32)       5376        ['input_10[0][0]']               
                                                                                                  
 lstm_43 (LSTM)                 (None, 24, 32)       8320        ['lstm_42[0][0]']                
                                                                                                  
 lstm_44 (LSTM)                 (None, 24, 32)       5376        ['input_10[0][0]']               
                                                                                            

In [186]:
model.fit(train_X, train_y, validation_split=0.2, epochs=50)

Epoch 1/50


2025-01-17 15:33:37.259640: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-17 15:33:37.630493: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-17 15:33:37.644054: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-17 15:33:37.876217: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-17 15:33:37.887320: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-17 15:33:38.143897: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-17 15:33:38.323613: I tensorflow/core/grappler/optimizers/cust

50/50 [==============================] - ETA: 0s - loss: 0.6190 - accuracy: 0.6949 - recall_6: 0.9766 - precision_6: 0.7046

2025-01-17 15:33:45.848596: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-17 15:33:45.982001: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-17 15:33:45.990759: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-17 15:33:46.206338: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-17 15:33:46.214657: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-17 15:33:46.384053: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-01-17 15:33:46.556135: I tensorflow/core/grappler/optimizers/cust

50/50 [==============================] - 12s 138ms/step - loss: 0.6190 - accuracy: 0.6949 - recall_6: 0.9766 - precision_6: 0.7046 - val_loss: 0.9854 - val_accuracy: 0.2970 - val_recall_6: 1.0000 - val_precision_6: 0.2970
Epoch 2/50
50/50 [==============================] - 3s 51ms/step - loss: 0.6087 - accuracy: 0.7050 - recall_6: 1.0000 - precision_6: 0.7050 - val_loss: 1.0579 - val_accuracy: 0.2970 - val_recall_6: 1.0000 - val_precision_6: 0.2970
Epoch 3/50
50/50 [==============================] - 3s 52ms/step - loss: 0.6143 - accuracy: 0.7050 - recall_6: 1.0000 - precision_6: 0.7050 - val_loss: 0.9661 - val_accuracy: 0.2970 - val_recall_6: 1.0000 - val_precision_6: 0.2970
Epoch 4/50
50/50 [==============================] - 2s 50ms/step - loss: 0.6083 - accuracy: 0.7050 - recall_6: 1.0000 - precision_6: 0.7050 - val_loss: 0.9082 - val_accuracy: 0.2970 - val_recall_6: 1.0000 - val_precision_6: 0.2970
Epoch 5/50
50/50 [==============================] - 3s 50ms/step - loss: 0.6087 - acc

50/50 [==============================] - 2s 49ms/step - loss: 0.4034 - accuracy: 0.8080 - recall_6: 0.8638 - precision_6: 0.8638 - val_loss: 0.6969 - val_accuracy: 0.6650 - val_recall_6: 0.7436 - val_precision_6: 0.4603
Epoch 37/50
50/50 [==============================] - 2s 49ms/step - loss: 0.4020 - accuracy: 0.8067 - recall_6: 0.8539 - precision_6: 0.8696 - val_loss: 0.6739 - val_accuracy: 0.6624 - val_recall_6: 0.8034 - val_precision_6: 0.4608
Epoch 38/50
50/50 [==============================] - 2s 49ms/step - loss: 0.3885 - accuracy: 0.8175 - recall_6: 0.8729 - precision_6: 0.8689 - val_loss: 0.7124 - val_accuracy: 0.6497 - val_recall_6: 0.7350 - val_precision_6: 0.4456
Epoch 39/50
50/50 [==============================] - 2s 48ms/step - loss: 0.3992 - accuracy: 0.8042 - recall_6: 0.8494 - precision_6: 0.8698 - val_loss: 0.6774 - val_accuracy: 0.6675 - val_recall_6: 0.8120 - val_precision_6: 0.4657
Epoch 40/50
50/50 [==============================] - 2s 49ms/step - loss: 0.4032 - a

In [187]:
loss, accuracy,_,_ = model.evaluate(test_X, test_y)
print(f"테스트 정확도: {accuracy*100:.2f}%")

27/27 [==============================] - 1s 46ms/step - loss: 0.6133 - accuracy: 0.7119 - recall_6: 0.5744 - precision_6: 0.5000
테스트 정확도: 71.19%


In [193]:
y_hat = model.predict(test_X)

27/27 [==============================] - 1s 20ms/step


In [194]:
y_hat = (y_hat>=0.5).astype('int')

In [195]:
f1 = f1_score(test_y.reshape(-1), y_hat.reshape(-1))
print(f"F1 스코어: {f1:.4f}")

F1 스코어: 0.5346


In [196]:
cross_tab = pd.crosstab(test_y.reshape(-1), y_hat.reshape(-1))
cross_tab

col_0,0,1
row_0,,
0,459,139
1,103,139


In [192]:
len(test_y), len(y_hat)

(840, 840)